# Web Scraping con Python

Este script muestra algunas posibilidades de la biblioteca *Selenium* que nos permite obtener, automáticamente, datos de páginas web dinámicas, es decir que necesitan pulsar botones, etc, etc.

Selenium está pensado realmente para automatizar pruebas de entornos web, pero a nosotros nos servirá para nuestro propósito de hacer web scraping.

En particular, lo que vamos a hacer es arrancar google Chrome y manejarlo automáticamente desde Python.

Para ello necesitamos 3 cosas:

1.- Tener google Chrome

2.- Instalar la biblioteca selenium

3.- Tener un fichero controlador, para uso utilizaremos la librería chromedriver_autoinstaller



Empezamos asegurarnos de que Selenium está instalado

In [ ]:
modules = ["selenium","chromedriver_autoinstaller"]


import sys
import os.path
from subprocess import check_call
import importlib
import os

def instala(modules):
    print("Instalando módulos")
    for m in modules:
        # para el import quitamos [...] y ==...
        p = m.find("[")
        mi = m if p==-1 else m[:p]
        p = mi.find("==")
        mi = mi if p==-1 else mi[:p]
        torch_loader = importlib.util.find_spec(mi)
        if torch_loader is not None:
            print(m," encontrado")
        else:
            print(m," No encontrado, instalando...",end="")  
            try:        
                r = check_call([sys.executable, "-m", "pip", "install", "--user",  m])
                print("¡hecho!")
            except:
                print("¡Problema al instalar ",m,"! ¿seguro que el módulo existe?",sep="")

    print("¡Terminado!")

instala(modules)  


Ahora abrimos el navegador

In [ ]:

import sys

import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import chromedriver_autoinstaller

# setup chrome options
chrome_options = webdriver.ChromeOptions()
#chrome_options.add_argument('--headless') # ensure GUI is off
#chrome_options.add_argument('--no-sandbox')
#chrome_options.add_argument('--disable-dev-shm-usage')

# set path to chromedriver as per your configuration
chromedriver_autoinstaller.install()


# set up the webdriver
driver = webdriver.Chrome(options=chrome_options)

# Banco de Santander

In [ ]:
url = 'https://www.investing.com/equities/banco-santander'
driver.get(url)



Simulamos un click en la página para aceptar las cookies


In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time

try:
    cookies = driver.find_element(By.ID, "onetrust-accept-btn-handler")
    cookies.click()
    print("Cookies aceptadas.")
except Exception as e:
    print("No se encontró el botón de aceptar cookies o ocurrió un error:", e)


In [ ]:

try:
    elemento = driver.find_element(By.CSS_SELECTOR, '[data-test="instrument-price-last"]')
except Exception as e:
    print("No encuentro el elemento con el último precio:", e)


In [ ]:
elemento.text

Obtenemos el elemento que incluye toda la tabla

In [ ]:
try:
    tabla = driver.find_element(By.CSS_SELECTOR, '[data-test="key-info"]')
except Exception as e:
    print("No se encontró el elemento tabla", e)

In [ ]:
tabla.text

In [ ]:
try:
    clase = "key-info_dd-numeric__ZQFIs"
    els = tabla.find_elements(By.CLASS_NAME, clase)
except Exception as e:
    print("No se encontró el elemento:", e)
        

In [ ]:
el = els[0]
print(el.text)

el=els[1]
print(el.text)

el=els[2]
print(el.text)

In [ ]:
valores = []
for el in els:
    valores.append(el.text) # lo añadimos a la lista
len(valores),valores    

Ahora los nombres de los datos 

In [ ]:
nombres = ['Prev. Close',
  'Open',
  "Day's Range 1",
  "Day's Range 2",
  '52 wk Range 1',
  '52 wk Range 2',
  'Volume',
  'Average Vol. (3m)',
  '1-Year Change',
   'Market Cap',
  'Shares Outstanding',
  'Revenue',
  'P/E Ratio',
  'EPS',
  'Dividend (Yield) 1',
  'Dividend (Yield) 2',
  'Beta']
    
import pandas as pd
df = pd.DataFrame([valores[0:len(nombres)]], columns=nombres)
df

In [ ]:
df.to_excel("santander.xlsx", index=False)
driver.close()

### IBEX 


In [ ]:

# setup chrome options
#chrome_options = webdriver.ChromeOptions()
#chrome_options.add_argument('--headless') # ensure GUI is off
#chrome_options.add_argument('--no-sandbox')
#chrome_options.add_argument('--disable-dev-shm-usage')

# set path to chromedriver as per your configuration
#chromedriver_autoinstaller.install()


# set up the webdriver
driver = webdriver.Chrome(options=chrome_options)

url = 'https://es.investing.com/indices/spain-35'
driver.get(url)
          

In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time

try:
    cookies = driver.find_element(By.ID, "onetrust-accept-btn-handler")
    cookies.click()
    print("Cookies aceptadas.")
except Exception as e:
    print("No se encontró el botón de aceptar cookies o ocurrió un error:", e)

In [ ]:

try:
    elemento = driver.find_element(By.CSS_SELECTOR, '[data-test="most-active-stocks-table"]')
    table = elemento.find_element(By.TAG_NAME, "table")
    print("Tabla seleccionada")
except Exception as e:
    print("No se encontró el elemento:", e)
            

Vamos fila a fila

In [ ]:

try:
    filas = table.find_elements(By.TAG_NAME, "tr") # elements porque hay más de 1
    print(len(filas))
except Exception as e:
    print("No se encontró el elemento:", e)

Cabeceras

In [ ]:
cabecera_els = filas[0].find_elements(By.TAG_NAME, "th")
cabecera = []
for cab in cabecera_els:
    cabecera.append(cab.text)
len(cabecera),cabecera    

Echamos un vistazo a las filas 1 a 1

In [ ]:
tabla = []
for i,fila in enumerate(filas[1:]):
    print("Fila ",i,end=" ")
    fdatos = []
    try:
        cols = fila.find_elements(By.TAG_NAME, "td") # elements porque hay más de 1
        print("Columnas ",len(cols))
        for j,col in enumerate(cols):
            print("Columna ",j,col.text)
            fdatos.append(col.text)
    except Exception as e:
        print("No se encontró el elemento:", e)
    tabla.append(fdatos)
    print("="*100)

In [ ]:
tabla

In [ ]:
columnas = ['Nombre',
  'Último',
  'Anterior',
  'Máximo',
  'Mínimo',
  '% Var.',
  'Vol.',
  'Abreviatura',          
  'Fecha' ]

Convertimos en un dataframe

In [ ]:
df = pd.DataFrame(tabla,columns = columnas)

df

In [ ]:
df[["Abreviatura","Nombre"]] = df['Nombre'].str.split('\n', expand=True)
df

In [ ]:
df.to_excel("activos.xlsx",index=False)